# BLU02 - Exercises Notebook

In [1]:
import hashlib # for grading

import os
import pandas as pd
import numpy as np
import datetime
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

## 1 Read the Programs data (graded)

In this first exercise, we aim to create a single dataframe, combining all programs from all seasons.

With a caveat though: **we want to include seasons after 1950**.

In [2]:
def read_season(folder_path, file_name):
    path = os.path.join(folder_path, file_name)
    return pd.read_csv(path)

def read_programs(folder_path):
    files = os.listdir(folder_path)
    # Create a list with the name of all files containing programs from
    # 1950 inclusive and onwards (just the filename, no complete path.)
    # files_after_1950: List[str] = ...
    files_after_1950 = list(filter(lambda x: int(x[0:4])>=1950, files))
    
    # Create a list with the dataframes
    # seasons: List[pd.DataFrame] = ...
    # YOUR CODE HERE
    seasons = [read_season(folder_path, file_name) for file_name in files_after_1950]
    
    # Use pd.concat to create a single dataframe.
    # programs: pd.DataFrame = ...
    #
    programs = pd.concat(seasons, axis=0, ignore_index=True)
    # Drop the column GUID.
    programs = (programs.drop("GUID", axis=1)
                .drop_duplicates()
                .set_index("ProgramID"))
    
    ## Remove Duplicated lines.
    
    # Set the index to be the column ProgramID
    # YOUR CODE HERE
    return programs

programs = read_programs(os.path.join('data', 'programs'))

In [3]:
assert programs['Season'].min() == '1950-51'
assert programs['Season'].max() == '2016-17'
assert programs.index.name == 'ProgramID'
assert programs.shape == (7341, 2)

In [4]:
programs.head()

,Orchestra,Season
ProgramID,,
9061,New York Philharmonic,2003-04
9060,New York Philharmonic,2003-04
9069,New York Philharmonic,2003-04
9285,Musicians from the New York Philharmonic,2003-04
9077,New York Philharmonic,2003-04


## 2 Read the Concerts data (graded)

Although we list all transformations step-by-step for the sake of clarity, we expect you to use method chaining.

In [5]:
def make_concerts(file_path): 
    # Read concerts data and drop the GUID column.
    # concerts: pd.DataFrame = ...
    # YOUR CODE HERE

    # Remember to_datetime? We need here. We need to parse the columns Date and 
    # Time. Use pd.to_datetime(...).dt.date for the Date and pd_to_datetime(..., 
    # format=%I:%M%p).dt.time for the Time.
    # YOUR CODE HERE

    ## Remove Duplicated lines.
    # YOUR CODE HERE

    concerts = pd.read_csv(file_path)
    
    concerts = (concerts.drop("GUID", axis=1))
    concerts["Date"] = pd.to_datetime(concerts["Date"]).dt.date
    concerts["Time"] = pd.to_datetime(concerts["Time"], format="%I:%M%p").dt.time
    concerts = concerts.drop_duplicates()
    return concerts

concerts = make_concerts(os.path.join('data','concerts.csv'))

In [6]:
concerts.head()

,ProgramID,ConcertID,EventType,Location,Venue,Date,Time
0,3853,0,Subscription Season,"Manhattan, NY",Apollo Rooms,1842-12-07,20:00:00
1,5178,0,Subscription Season,"Manhattan, NY",Apollo Rooms,1843-02-18,20:00:00
2,10785,0,Special,"Manhattan, NY",Apollo Rooms,1843-04-07,20:00:00
3,5887,0,Subscription Season,"Manhattan, NY",Apollo Rooms,1843-04-22,20:00:00
4,305,0,Subscription Season,"Manhattan, NY",Apollo Rooms,1843-11-18,NaT


In [7]:
assert concerts.shape == (21582, 7)
assert concerts.Date.min() == datetime.date(1842, 12, 7)
assert concerts.Date.max() == datetime.date(2017, 7, 7)
assert set(concerts.columns) == set([
    'ProgramID', 'ConcertID', 'EventType', 'Location', 'Venue', 'Date', 'Time'
])

## 3 Combine Programs and Concerts data (graded)

Let's combine both dataframes into a single dataset, using an inner join.

In [8]:
# Remember that you want to join on the index of one of the dataframes.
# Join only the concerts with valid ProgramIDs
# nyp = ...
# YOUR CODE HERE
nyp = pd.merge(programs, concerts, left_index = True, right_on = "ProgramID")

In [9]:
nyp.head()

,Orchestra,Season,ProgramID,ConcertID,EventType,Location,Venue,Date,Time
12038,New York Philharmonic,2003-04,9061,0,Subscription Season,"Manhattan, NY",Avery Fisher Hall,2003-09-17,19:30:00
12039,New York Philharmonic,2003-04,9060,0,Subscription Season,"Manhattan, NY",Avery Fisher Hall,2003-09-18,19:30:00
17642,New York Philharmonic,2003-04,9060,1,Subscription Season,"Manhattan, NY",Avery Fisher Hall,2003-09-19,20:00:00
19884,New York Philharmonic,2003-04,9060,2,Subscription Season,"Manhattan, NY",Avery Fisher Hall,2003-09-20,20:00:00
21280,New York Philharmonic,2003-04,9060,3,Subscription Season,"Manhattan, NY",Avery Fisher Hall,2003-09-23,19:30:00


In [10]:
assert nyp.shape == (13254, 9)
assert set(nyp.columns) == set([
    'ProgramID', 'ConcertID', 'EventType', 'Location', 'Venue',
    'Date', 'Time', 'Orchestra', 'Season'
])

## 4 Read Works and Soloists data (graded)

We will read the two remaining pieces of data. 

Again, albeit the step-by-step description, we encourage you to use method chaining.

In [11]:
def subset(df, mask):
    return df[mask]

def read_works(file_path):
    # Read the works data.
    # works: pd.DataFrame = ...
    # YOUR CODE HERE

    # Remove the Intervals (attention to the values in the isInterval column).
    # works: pd.DataFrame = ...
    # YOUR CODE HERE

    # Select the columns ProgramID, WorkID, ComposerName, WorkTitle, Movement and ConductorName.
    # YOUR CODE HERE

    ## Remove Duplicated lines.
    # YOUR CODE HERE

    
    works = pd.read_csv(file_path)
    works = (works.pipe(subset, works.isInterval==False)
            .filter(["ProgramID", "WorkID", "ComposerName", "WorkTitle", "Movement", "ConductorName"])
            .drop_duplicates())
    
    return works


def read_soloists(file_path):
    # Read the soloists data and drop GUID and MovementID Columns
    # YOUR CODE HERE

    ## Remove Duplicated lines.
    # YOUR CODE HERE
    soloists = pd.read_csv(file_path)
    
    soloists = (soloists.drop(["GUID", "MovementID"], axis=1)
                .drop_duplicates())

    return soloists


works = read_works('data/works.csv')
soloists = read_soloists('data/soloists.csv')

In [12]:
soloists

,ProgramID,WorkID,SoloistName,SoloistInstrument,SoloistRole
0,2117,4452,"Timm, Henry C.",Piano,A
1,4331,51060,Arion Choral Society,Mens Chorus,S
2,5080,51057,Chorus,Boys Choir,S
3,11474,1587,"Essipoff, Annette",Piano,S
4,11600,53976,Oratorio Society of New York,Chorus,S
...,...,...,...,...,...
55534,10683,11754,"Ellis, Kenny",Special Guest,A
55546,12009,8293,"Keillor, Garrison",Vocalist,S
55548,12009,8293,"DiGiallonardo , Christine",Vocalist,S
55550,12009,8293,"Dworsky, Richard",Piano,S


In [13]:
works

,ProgramID,WorkID,ComposerName,WorkTitle,Movement,ConductorName
0,3853,52446,"Beethoven, Ludwig van","SYMPHONY NO. 5 IN C MINOR, OP.67",NaN,"Hill, Ureli Corelli"
1,5178,52437,"Beethoven, Ludwig van","SYMPHONY NO. 3 IN E FLAT MAJOR, OP. 55 (EROICA)",NaN,"Hill, Ureli Corelli"
2,10785,52364,"Beethoven, Ludwig van","EGMONT, OP.84",Overture,"Hill, Ureli Corelli"
3,5887,52434,"Beethoven, Ludwig van","SYMPHONY NO. 2 IN D MAJOR, OP.36",NaN,"Boucher, Alfred"
4,305,52453,"Beethoven, Ludwig van","SYMPHONY NO. 7 IN A MAJOR, OP.92",NaN,"Hill, Ureli Corelli"
...,...,...,...,...,...,...
82571,9678,53976,"Handel, George Frideric",MESSIAH,Chorus: Worthy is the Lamb that was slain,"McGegan, Nicholas"
82572,10608,53976,"Handel, George Frideric",MESSIAH,Chorus: Worthy is the Lamb that was slain,"Labadie, Bernard"
82573,9542,53976,"Handel, George Frideric",MESSIAH,Chorus: Amen,"Bicket, Harry"
82574,9678,53976,"Handel, George Frideric",MESSIAH,Chorus: Amen,"McGegan, Nicholas"


In [14]:
assert works.shape == (71065, 6)
assert set(works.columns) == set([
    'ProgramID', 'WorkID', 'ComposerName', 'WorkTitle', 'Movement', 'ConductorName'
])

assert soloists.shape == (50292, 5)
assert set(soloists.columns) == set([
   'ProgramID', 'WorkID', 'SoloistName', 'SoloistInstrument', 'SoloistRole'
])

## 5 Combine Works and Soloists (graded)

Like we did for Programs and Concerts, now we combine Works and Soloists.

In [15]:
# Combine both dataframes, again using an inner type of join. An work is identified by the pair
# ProgramId, WorkID
# works_and_soloists : pd.DataFrame = ....
# YOUR CODE HERE
works_and_soloists = pd.merge(works, soloists)

In [16]:
assert works_and_soloists.shape == (64698, 9)
assert set(works_and_soloists.columns) == set(
    [
        'ProgramID', 'WorkID', 'ComposerName', 'WorkTitle', 'Movement',
        'ConductorName', 'SoloistName', 'SoloistInstrument', 'SoloistRole'
    ]
)

In [17]:
nyp.head()

,Orchestra,Season,ProgramID,ConcertID,EventType,Location,Venue,Date,Time
12038,New York Philharmonic,2003-04,9061,0,Subscription Season,"Manhattan, NY",Avery Fisher Hall,2003-09-17,19:30:00
12039,New York Philharmonic,2003-04,9060,0,Subscription Season,"Manhattan, NY",Avery Fisher Hall,2003-09-18,19:30:00
17642,New York Philharmonic,2003-04,9060,1,Subscription Season,"Manhattan, NY",Avery Fisher Hall,2003-09-19,20:00:00
19884,New York Philharmonic,2003-04,9060,2,Subscription Season,"Manhattan, NY",Avery Fisher Hall,2003-09-20,20:00:00
21280,New York Philharmonic,2003-04,9060,3,Subscription Season,"Manhattan, NY",Avery Fisher Hall,2003-09-23,19:30:00


## 6 Combine everything (graded)

The final goal here is to create a single dataframe.

In [18]:
# Combine works_and_soloists and nyp into a single dataframe.
# nyp_merged = ...
nyp_merged = pd.merge(nyp, works_and_soloists, on = "ProgramID")

In [19]:
assert nyp_merged.shape == (72185, 17)
assert set(nyp_merged.columns) == set(
    [
       'ProgramID', 'ConcertID', 'EventType', 'Location', 'Venue', 'Date',
       'Time', 'Orchestra', 'Season', 'WorkID', 'ComposerName', 'WorkTitle',
       'Movement', 'ConductorName', 'SoloistName', 'SoloistInstrument',
       'SoloistRole'
    ]
)

In [20]:
nyp_merged.head()

,Orchestra,Season,ProgramID,ConcertID,EventType,Location,Venue,Date,Time,WorkID,ComposerName,WorkTitle,Movement,ConductorName,SoloistName,SoloistInstrument,SoloistRole
0,New York Philharmonic,2003-04,9061,0,Subscription Season,"Manhattan, NY",Avery Fisher Hall,2003-09-17,19:30:00,7534,"Verdi, Giuseppe",SIMON BOCCANEGRA,"""Il lacerato spirito"" (Prologue), Jacopo Fiesc...","Maazel, Lorin","Ramey, Samuel",Bass,S
1,New York Philharmonic,2003-04,9061,0,Subscription Season,"Manhattan, NY",Avery Fisher Hall,2003-09-17,19:30:00,52071,"Verdi, Giuseppe",DON CARLO,"""Ella giammai m'amò,"" King Philip II (aria), A...","Maazel, Lorin","Ramey, Samuel",Bass,S
2,New York Philharmonic,2003-04,9061,0,Subscription Season,"Manhattan, NY",Avery Fisher Hall,2003-09-17,19:30:00,1810,"Verdi, Giuseppe",ERNANI,"""Infelice! e tuo credevi,"" Silva, Act I, scene ix","Maazel, Lorin","Ramey, Samuel",Bass,S
3,New York Philharmonic,2003-04,9060,0,Subscription Season,"Manhattan, NY",Avery Fisher Hall,2003-09-18,19:30:00,8155,"Hartke, Stephen",SYMPHONY NO. 3,NaN,"Maazel, Lorin",Hilliard Ensemble,Ensemble,S
4,New York Philharmonic,2003-04,9060,0,Subscription Season,"Manhattan, NY",Avery Fisher Hall,2003-09-18,19:30:00,8155,"Hartke, Stephen",SYMPHONY NO. 3,NaN,"Maazel, Lorin","James, David",Countertenor,A


## 7 Final transformations (graded)

Now, we perform the train-test split.

We also perform some final transformations on both datasets:

* Include some date features: Year, Month, Hour, Minute, Day and Weekday
* Create a new feature, ComposerLastName from ComposerName column. 
* Filter out rows that have a location that appear is less than 10 times in the DataFrame.
* Drop ProgramID, ConcertID, WorkID, Date and Season

In [21]:
def append_date_features(df):
    df[['Year','Month', "Day", "Weekday"]] = pd.DataFrame([(x.year, x.month, x.day, x.weekday()) for x in df['Date']])
    df[['Hour','Minute']] = pd.DataFrame([(x.hour, x.minute) for x in df['Time']])
    return df

def append_composer_last_name(df):
    # YOUR CODE HERE
    df["ComposerLastName"] = df.ComposerName.str.split(",", expand=True)[0]
    return df

def preprocess_data(df):
    # You should follow these exact steps:
    #   1 - Include some date features: Year, Month, Hour, Minute, Day and Weekday
    #   2 - Create a new feature, ComposerLastName from ComposerName column. 
    #   3 - Filter out rows that have a location that appear is less than 10 times in the DataFrame.
    #   4 - Drop ProgramID, ConcertID, WorkID, Season, Date, Time
    #   
    # YOUR CODE HERE
    df = (df.pipe(append_date_features)
          .pipe(append_composer_last_name)
          .groupby('Location').filter(lambda x: x.shape[0] >= 10)
          .drop(["ProgramID", "ConcertID", "WorkID", "Season", "Date", "Time"], axis=1))
    return df


nyp_preprossed = preprocess_data(nyp_merged)

In [22]:
nyp_preprossed

,Orchestra,EventType,Location,Venue,ComposerName,WorkTitle,Movement,ConductorName,SoloistName,SoloistInstrument,SoloistRole,Year,Month,Day,Weekday,Hour,Minute,ComposerLastName
0,New York Philharmonic,Subscription Season,"Manhattan, NY",Avery Fisher Hall,"Verdi, Giuseppe",SIMON BOCCANEGRA,"""Il lacerato spirito"" (Prologue), Jacopo Fiesc...","Maazel, Lorin","Ramey, Samuel",Bass,S,2003,9,17,2,19.0,30.0,Verdi
1,New York Philharmonic,Subscription Season,"Manhattan, NY",Avery Fisher Hall,"Verdi, Giuseppe",DON CARLO,"""Ella giammai m'amò,"" King Philip II (aria), A...","Maazel, Lorin","Ramey, Samuel",Bass,S,2003,9,17,2,19.0,30.0,Verdi
2,New York Philharmonic,Subscription Season,"Manhattan, NY",Avery Fisher Hall,"Verdi, Giuseppe",ERNANI,"""Infelice! e tuo credevi,"" Silva, Act I, scene ix","Maazel, Lorin","Ramey, Samuel",Bass,S,2003,9,17,2,19.0,30.0,Verdi
3,New York Philharmonic,Subscription Season,"Manhattan, NY",Avery Fisher Hall,"Hartke, Stephen",SYMPHONY NO. 3,NaN,"Maazel, Lorin",Hilliard Ensemble,Ensemble,S,2003,9,18,3,19.0,30.0,Hartke
4,New York Philharmonic,Subscription Season,"Manhattan, NY",Avery Fisher Hall,"Hartke, Stephen",SYMPHONY NO. 3,NaN,"Maazel, Lorin","James, David",Countertenor,A,2003,9,18,3,19.0,30.0,Hartke
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72180,New York Philharmonic,Parks,"Staten Island, NY",Clove Lake Park,"Kraft, William","CONCERTO, FOUR PERCUSSION SOLOISTS",NaN,"Lewis, Henry","Kohloff, Roland",Percussion,S,1973,7,28,5,20.0,30.0,Kraft
72181,New York Philharmonic,Parks,"Staten Island, NY",Clove Lake Park,"Kraft, William","CONCERTO, FOUR PERCUSSION SOLOISTS",NaN,"Lewis, Henry","Rosenberger, Walter",Percussion,S,1973,7,28,5,20.0,30.0,Kraft
72182,New York Philharmonic,Parks,"Staten Island, NY",Clove Lake Park,"Kraft, William","CONCERTO, FOUR PERCUSSION SOLOISTS",NaN,"Lewis, Henry","Lang, Morris ""Arnie""",Percussion,S,1973,7,28,5,20.0,30.0,Kraft
72183,New York Philharmonic,Parks,"Staten Island, NY",Clove Lake Park,"Kraft, William","CONCERTO, FOUR PERCUSSION SOLOISTS",NaN,"Lewis, Henry","Fitz, Richard",Percussion,S,1973,7,28,5,20.0,30.0,Kraft


In [23]:
assert nyp_preprossed.shape == (71659, 18)
assert set(nyp_preprossed.columns) == {
       'EventType', 'Location', 'Venue', 'Orchestra',
       'ComposerName', 'WorkTitle', 'Movement', 'ConductorName', 'SoloistName',
       'SoloistInstrument', 'SoloistRole', 'Year', 'Month', 'Day', 'Hour',
       'Minute', 'Weekday', 'ComposerLastName'
}
assert nyp_preprossed.groupby('Location').size().min() == 10
assert nyp_preprossed.ComposerLastName.value_counts().loc['Mozart'] == 3333
assert nyp_preprossed.ComposerLastName.value_counts().loc['Gershwin'] == 2127
assert nyp_preprossed.ComposerLastName.nunique() == 1241

# The house prices dataset

A dataset containing several characteristics of several houses and their sell price 

* LotFrontage: Linear feet of street connected to property
* LotArea: Lot size in square feet
* OverallQual: Rates the overall material and finish of the house
       10  Very Excellent
       9	Excellent
       8	Very Good
       7	Good
       6	Above Average
       5	Average
       4	Below Average
       3	Fair
       2	Poor
       1	Very Poor
* OverallCond: Rates the overall condition of the house

       10	Very Excellent
       9	Excellent
       8	Very Good
       7	Good
       6	Above Average	
       5	Average
       4	Below Average	
       3	Fair
       2	Poor
       1	Very Poor
* MasVnrArea: Masonry veneer area in square feet
* BsmtFinSF1: Type 1 finished square feet
* BsmtUnfSF: Unfinished square feet of basement area
* TotalBsmtSF: Total square feet of basement area
* 1stFlrSF: First Floor square feet
* 2ndFlrSF: Second floor square feet
* LowQualFinSF: Low quality finished square feet (all floors)
* GrLivArea: Above grade (ground) living area square feet
* BsmtFullBath: Basement full bathrooms
* BsmtHalfBath: Basement half bathrooms
* FullBath: Full bathrooms above grade
* HalfBath: Half baths above grade
* BedroomAbvGr: Bedrooms above grade (does NOT include basement bedrooms)
* KitchenAbvGr: Kitchens above grade
* TotRmsAbvGrd: Total rooms above grade (does not include bathrooms)
* Fireplaces: Number of fireplaces
* GarageCars: Size of garage in car capacity
* GarageArea: Size of garage in square feet
* WoodDeckSF: Wood deck area in square feet
* OpenPorchSF: Open porch area in square feet
* EnclosedPorch: Enclosed porch area in square feet
* 3SsnPorch: Three season porch area in square feet
* ScreenPorch: Screen porch area in square feet
* PoolArea: Pool area in square feet
* MiscVal: $Value of miscellaneous feature 
* SellingDate: Date when the house was sold
* BuildingDate: Date when the house was built
* RemodAddDate: Remodel date (same as construction date if no remodeling or additions)
* SalePrice: The house price at the selling date (our target variable)

In [24]:
def house_price_dataset():
    return pd.read_csv(
    'data/housePrices.csv', 
        parse_dates=[
            'SellingDate',
            'BuildingDate',
            'RemodAddDate'
        ]
    )

dataset = house_price_dataset()
dataset_train, dataset_test = train_test_split(dataset, random_state=0)
X_train = dataset_train.drop(columns='SalePrice')
y_train = dataset_train.SalePrice
X_test = dataset_test.drop(columns='SalePrice')
y_test = dataset_test.SalePrice

## 8 Build a DateTransformer transformer (graded)

There's a simple transformer that can be useful, from times to times, when modeling.

What we want is to build a transformer that transform dates into timedeltas.

Usually when you have features that are Dates you compute a time delta between the feature and a given refence date.

e.g Imagine that your clients have a loyalty period that ends at a given date. When your model is doing some predictions, one of the features that you can use is the number of days until the end of the loyalty period. i.e the date when the loyalty ends minus the date when your model is running. 

In the house prices dataset, the selling date will be the reference data, since we want to predict the house price at the selling date. Then we need to convert the other date using our transformer

In [25]:
class DateTransformer(BaseEstimator, TransformerMixin):
    # Implement the __init__ method.
    # Our DateTransformer must be able to receive two parameters: 
    # datetime_cols: a list, that contains the datetime cols that should be converted
    # ref_date_col - indicates the name of the column that should be used as reference date,
    # YOUR CODE HERE
    def __init__(self, datetime_cols, ref_date_col):
        self.datetime_cols = datetime_cols
        self.ref_date_col = ref_date_col
        
    # There's no need for a fit method in this case, it does nothing.
    # We should be able to call fit without any explicit parameters.
    # Meaning: we should be able to call transformer.fit().
    # YOUR CODE HERE
    
    def fit(self, X, y=None):
        # Fit the transformer and store it.
        return self

    # Transform should transform all datetime columns into the difference in days to the reference date.
    # The reference date column should be dropped. 
    # YOUR CODE HERE
    
    def transform(self, X):
        X = pd.DataFrame(X)
        X = X.copy()
        for column in self.datetime_cols:
            X[column] = X[column] - X[self.ref_date_col]
            X[column] = [x.days for x in X[column]]
        X.drop(self.ref_date_col, axis=1, inplace=True)
        
        return X

In [26]:
X_train_trasformed = DateTransformer(
    datetime_cols=['BuildingDate', 'RemodAddDate'], 
    ref_date_col='SellingDate'
).fit_transform(X_train)
assert X_train_trasformed.BuildingDate.min() == -49008
assert X_train_trasformed.BuildingDate.max() == -1
assert 'SellingDate' not in X_train_trasformed.columns
assert X_train_trasformed.dtypes.BuildingDate == np.dtype('int64')
assert X_train_trasformed.dtypes.RemodAddDate == np.dtype('int64')

## 9 Building the pipeline (graded)

Finally, we want to use the two transformers together and run a linear regression on top.

We want to Covert hour dates to time deltas relative to the Selling Date.

We want to scale all features to the same range, using `sklearn.preprocessing.StandardScaler()`.

We want to estimate the SellingPrice using a Liner Regression.

Standardization of datasets is a common requirement for many machine learning estimators implemented in scikit-learn; they might behave badly if the individual features do not more or less look like standard normally distributed data: Gaussian with zero mean and unit variance.

In practice we often ignore the shape of the distribution and just transform the data to center it by removing the mean value of each feature, then scale it by dividing non-constant features by their standard deviation.

For instance, many elements used in the objective function of a learning algorithm (such as the RBF kernel of Support Vector Machines or the l1 and l2 regularizers of linear models) assume that all features are centered around zero and have variance in the same order. If a feature has a variance that is orders of magnitude larger than others, it might dominate the objective function and make the estimator unable to learn from other features correctly as expected.



In [27]:
# Create a pipeline including:
#   1 - 'date_converter', DateTransformer(['BuildingDate', 'RemodAddDate'], ref_date_col='SellingDate')
#   2 - 'standard_scaller', StandardScaler() with the default parameters
#   3 - 'model', LinearRegression
# YOUR CODE HERE

pipeline = Pipeline([('date_converter', DateTransformer(['BuildingDate', 'RemodAddDate'], ref_date_col='SellingDate')),
                     ('standard_scaller', StandardScaler()),
                    ('model', LinearRegression())])


pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
print('MAE: {}'.format(mae))

MAE: 20727.899453109087


In [28]:
assert type(pipeline) == Pipeline
assert type(pipeline.named_steps['date_converter']) == DateTransformer
assert type(pipeline.named_steps['standard_scaller']) == StandardScaler
assert pipeline.named_steps['date_converter'].get_params()['ref_date_col'] == 'SellingDate'
assert set(
    pipeline.named_steps['date_converter'].get_params()['datetime_cols']
) == {'BuildingDate', 'RemodAddDate'}
assert type(pipeline.named_steps['model']) == LinearRegression 

Exercises complete, congratulations! You are about to become a certified data wrangler.